<a href="https://colab.research.google.com/github/snig-17/QM-GRUNGE/blob/main/claire_peak_expeditions_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv("everest_expeditions.csv")

# Extract year from 'Yr/Seas'
df["Year"] = df["Yr/Seas"].str[:4].astype(int)

In [ ]:
exp_per_year = (
    df.groupby("Year")["Exped ID"]
      .nunique()
      .reset_index(name="Expeditions")
)


In [ ]:
df["Smtres"] = pd.to_numeric(df["Smtres"], errors="coerce")

exp_scale = (
    df.groupby("Year")["Smtres"]
      .sum(min_count=1)
      .reset_index(name="Total_Summiters")
)
